In [1]:
import pandas as pd
from pandas.io.json import json_normalize
# make a function to Preprocess the text data by removing punctuation, special characters, and stop words, and by performing stemming or lemmatization to 
# reduce each word to its base form. 

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import spacy

# import lancaster
from nltk.stem import LancasterStemmer
#import snowballStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nlp = spacy.load('en_core_web_sm')


# doc = nlp('I have purchased this product for my neighbors and as of now, no issues reported. So, I am very happy with this product')
# 
# BIGGEST USECASE -> Analyzing comments under youtube product review videos
# using spacy
class sentiment_analyzer:
# CHANGE REWARD_VALUE TO HELPFULNESS RATING AND STAR RATING AS ACTION AND MAKE ALPHA CONSTANT AND TRY AGAIN

    def __init__(self, reviewsFilePath):
        self.qTable = {}
        self.trialSet = []
        self.discount_rate_GAMMA = 0.7
        df = pd.read_json(reviewsFilePath, lines=True)
        # for index, row in df["reviewText"].head(10).iteritems():
        for (index, row), (reviewIndex, reviewRow), (starIndex, starRow) in zip(df["helpful"].head(5000).iteritems(), df["reviewText"].head(5000).iteritems(), df["overall"].head(5000).iteritems()):
            # Here we will only take data that people either found helpful or not so we can better assign a reward to the product
            # We will use the helpful or not parameter to determine the learning rate
            positive_helpfulness_reviews = float(row[0])
            all_helpfulness_reviews = float(row[1])
            if all_helpfulness_reviews > 0:
                avg_helpfulness = positive_helpfulness_reviews/all_helpfulness_reviews
                reviewText = self.clean_text_spacy(reviewRow)
                learning_rate_ALPHA = -1
                if all_helpfulness_reviews > 0.1 and all_helpfulness_reviews <= 6:
                    learning_rate_ALPHA = 0.05 * avg_helpfulness
                elif all_helpfulness_reviews > 6 and all_helpfulness_reviews < 15:
                    learning_rate_ALPHA = 0.1 * avg_helpfulness
                elif all_helpfulness_reviews >= 15 and all_helpfulness_reviews < 20:
                    learning_rate_ALPHA = 0.15 * avg_helpfulness
                elif all_helpfulness_reviews >= 20:
                    learning_rate_ALPHA = 0.2 * avg_helpfulness

                # print(positive_helpfulness_reviews, "/", all_helpfulness_reviews, "->", learning_rate_ALPHA)
                reviewWords = reviewText.split()

                for word in reviewWords:
                    temp_tuple = (word, learning_rate_ALPHA, float(starRow), avg_helpfulness)
                    self.trialSet.append(temp_tuple)
                    temp_tuple_qtable = (word, float(starRow))
                    if temp_tuple_qtable not in self.qTable:
                        #Inserted tuple will be (average_learning_rate_ALPHA, qValue, number of occurences, avg_helpfulness)
                        self.qTable[temp_tuple_qtable] = [learning_rate_ALPHA, 0, 1, avg_helpfulness]
                    if temp_tuple_qtable in self.qTable:
                        oldValue = self.qTable[temp_tuple_qtable]
                        newAlpha = ((learning_rate_ALPHA + oldValue[0]) / oldValue[2])
                        totalOccurnces = oldValue[2] + 1
                        self.qTable[temp_tuple_qtable] = [newAlpha, 0, totalOccurnces, avg_helpfulness]

        for i in range(len(self.trialSet)):
            word, alpha, rewardStar, helpfulness = self.trialSet[i]
            if helpfulness > 0.69 and alpha >= 0.2:
                helpfulness *= 5
            elif helpfulness > 0.69 and alpha >= 0.15:
                helpfulness *= 4
            elif helpfulness > 0.69 and alpha >= 0.1:
                helpfulness *= 3
            elif helpfulness > 0.69 and alpha >= 0.1:
                helpfulness *= 2
            next_word, next_alpha, next_rewardStar, next_helpfulness = None, None, None, None
            # Checking if this is the last element of the array.
            if i + 1 < len(self.trialSet):
                next_word, next_alpha, next_rewardStar, next_helpfulness = self.trialSet[i+1]
            max_q = -float('inf')
            for (next_star) in [1.0, 2.0, 3.0, 4.0, 5.0]:
                if (next_word, next_star) in self.qTable:
                    qTable_arr = self.qTable[(next_word, next_star)]
                    max_q = max(max_q, qTable_arr[1])
            # self.qTable[(word, rewardStar)][1] = self.qTable[(word, rewardStar)][1] + ((alpha+self.qTable[(word, rewardStar)][0]) / 2) * (helpfulness + self.discount_rate_GAMMA * max_q - self.qTable[(word, rewardStar)][1])
            self.qTable[(word, rewardStar)][1] = self.qTable[(word, rewardStar)][1] + alpha * (helpfulness + self.discount_rate_GAMMA * max_q - self.qTable[(word, rewardStar)][1])
            # Allocate rewards
        # print(self.qTable)
    def clean_text_spacy(self, text):
        doc = nlp(text)
        doc = [token for token in doc if not token.is_stop and not token.is_punct]
        doc = [token.lemma_ for token in doc]
        doc = ' '.join(doc)
        # print(doc)
        return doc

    def clean_text(self, text):
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        ws = WordNetLemmatizer()
        text = [ws.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
        text = ' '.join(text)
        # print(text)
        return text


    def getRewardStar(self, reviewText):
        cleanedReviewText = self.clean_text_spacy(reviewText)
        cleanedReviewText_arr = cleanedReviewText.split()

        totalMaxStar = 0
        counter = 0
        for word in cleanedReviewText_arr:
            max_q = -float('inf')
            max_star = -1
            for star in [1.0, 2.0, 3.0, 4.0, 5.0]:
                if ((word, star) in self.qTable):
                    qVal = self.qTable[(word, star)][1]
                    print("(word, star): ", (word, star))
                    print("qVal: ", qVal)
                    if qVal > max_q:
                        max_q = qVal
                        max_star = star
            print("max_star: ", max_star)
            if max_star > -1:
                totalMaxStar += max_star
                counter += 1
        return totalMaxStar/counter

def main():
    analyzerObj = sentiment_analyzer("./Video_games_5.json")
    print(analyzerObj.getRewardStar("This is a very bad good awesome COOL fun happy horrible terrible hard difficult problem okay dog cat and shit product."))
        

if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(word, star):  ('bad', 1.0)
qVal:  2.8570780670314804
(word, star):  ('bad', 2.0)
qVal:  3.652169759296556
(word, star):  ('bad', 3.0)
qVal:  3.8701186938234904
(word, star):  ('bad', 4.0)
qVal:  3.2271933751813418
(word, star):  ('bad', 5.0)
qVal:  4.394833124661783
max_star:  5.0
(word, star):  ('good', 1.0)
qVal:  2.8623393802197308
(word, star):  ('good', 2.0)
qVal:  3.2094773661163756
(word, star):  ('good', 3.0)
qVal:  3.8808659875605698
(word, star):  ('good', 4.0)
qVal:  3.8667863137691585
(word, star):  ('good', 5.0)
qVal:  4.818109547446751
max_star:  5.0
(word, star):  ('awesome', 1.0)
qVal:  0.6717005174205426
(word, star):  ('awesome', 2.0)
qVal:  0.42859960632225236
(word, star):  ('awesome', 3.0)
qVal:  0.33429486441125494
(word, star):  ('awesome', 4.0)
qVal:  2.5206247469607233
(word, star):  ('awesome', 5.0)
qVal:  3.4859330582651555
max_star:  5.0
(word, star):  ('cool', 1.0)
qVal:  0.7587344671634545
(word, star):  ('cool', 2.0)
qVal:  1.9211679823232923
(word, star